In [2]:
import requests

S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

cats = [
    "Category:Russo-Ukrainian War",
    "Category:2010s conflicts",
    "Category:2020s conflicts",
    "Category:2010s in Russia",
    "Category:2010s in Ukraine",
    "Category:2020s in Russia",
    "Category:2020s in Ukraine",
    "Category:Conflicts in Ukraine",
    "Category:Invasions by Russia",
    "Category:Invasions of Ukraine",
    "Category:Wars involving Russia",
    "Category:Wars involving Ukraine",
    "Category:Conflicts in territory of the former Soviet Union",
    "Category:Russia–Ukraine military relations",
    "Category:Russian irredentism",
    "Category:Vladimir Putin",
    "Category:Petro Poroshenko",
    "Category:Volodymyr Zelenskyy",
    "Category:21st-century military history of Russia",
    "Category:21st-century military history of Ukraine"
]
articleTitles = []
for category in cats:
    PARAMS = {
        "action": "query",
        "cmtitle": category,
        "cmlimit": "100",
        "list": "categorymembers",
        "format": "json"
    }

    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()

    PAGES = DATA['query']['categorymembers']
    print(len(PAGES))
    
    for page in PAGES:
        articleTitles.append(page['title'])

correctTitles = [tit for tit in set(articleTitles) if not tit.startswith("Category:")]
print(correctTitles)

90
Russo-Ukrainian War
Annexation of Crimea by the Russian Federation
Russian invasion of Ukraine
War in Donbas
Outline of the Russo-Ukrainian War
2014 pro-Russian unrest in Ukraine
2015 Ukraine power grid hack
2024 visits by Viktor Orbán to Russia and China
Anti-Maidan
Best in Hell
Book of Remembrance for Those Who Fell for Ukraine
Bridges in the Russo-Ukrainian War
Confiscation of Russian central bank funds
Crimean consensus
Croatia and the Russo-Ukrainian War
2017 Ukraine ransomware attacks
Environmental impact of the Russian occupation of Crimea
European Deterrence Initiative
Georgia and the Russian invasion of Ukraine
Humanitarian impacts of the Russian invasion of Ukraine
Humanitarian situation during the war in Donbas
Hydraulic warfare
Intermarium (region)
International recognition of the Donetsk People's Republic and the Luhansk People's Republic
International sanctions during the Russian invasion of Ukraine
International sanctions during the Russo-Ukrainian War
Klintsy oil dep

In [3]:
import requests

S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

def fetch_all_revisions(titles):
    revisions_by_title = {}

    for title in titles:
        print(f"Fetching all revisions for: {title}")
        params = {
            "action": "query",
            "prop": "revisions",
            "titles": title,
            "rvprop": "timestamp|user",
            "rvslots": "main",
            "rvlimit": "max",
            "formatversion": "2",
            "format": "json"
        }

        revisions = []
        while True:
            response = S.get(url=URL, params=params)
            if response.status_code != 200:
                print(f"Error fetching revisions for {title}: HTTP {response.status_code}")
                break

            data = response.json()
            pages = data.get("query", {}).get("pages", [])

            for page in pages:
                revisions.extend(page.get("revisions", []))

            if "continue" in data:
                params.update(data["continue"])
            else:
                break

        revisions_by_title[title] = revisions

    return revisions_by_title

all_revisions = fetch_all_revisions(correctTitles)

Fetching all revisions for: Wagner Group–Russian Ministry of Defence conflict
Fetching all revisions for: Russo-Ukrainian cyberwarfare
Fetching all revisions for: 2019 Russia–Africa Summit
Fetching all revisions for: Yemeni crisis
Fetching all revisions for: We Are Together with Russia
Fetching all revisions for: Kiev Cossacks insurrection
Fetching all revisions for: Nyonoksa radiation accident
Fetching all revisions for: Ukrainian War of Independence
Fetching all revisions for: Russian invasion of Ukraine
Fetching all revisions for: Iran–Israel proxy conflict
Fetching all revisions for: 2023 Black Sea drone incident
Fetching all revisions for: Luna 27
Fetching all revisions for: 2020 in Russia
Fetching all revisions for: Russo-Persian War (1826–1828)
Fetching all revisions for: 2012 in Russia
Fetching all revisions for: Deluge (history)
Fetching all revisions for: Lukoil oil transit dispute
Fetching all revisions for: Tucker Carlson's interview with Vladimir Putin
Fetching all revisio

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

all_revisions_flat = []
for title, revisions in all_revisions.items():
    for rev in revisions:
        timestamp = rev["timestamp"]
        all_revisions_flat.append({"title": title, "timestamp": timestamp})

df = pd.DataFrame(all_revisions_flat)

df["timestamp"] = pd.to_datetime(df["timestamp"])

df = df[df["timestamp"] >= "2020-01-01"]

df["month"] = df["timestamp"].dt.to_period("M")

monthly_revisions = df.groupby("month").size().reset_index(name="revision_count")

plt.figure(figsize=(14, 7))
plt.plot(monthly_revisions["month"].astype(str), monthly_revisions["revision_count"], marker='o')
plt.title("Monthly Revision Count Across All Pages (2020 and Later)")
plt.xlabel("Month")
plt.ylabel("Number of Revisions")
plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()
plt.show()
